# Loops

In [3]:
#credo che scriviamo una funzione che genera dei numeri casuali e calcola la media. credo che l obiettivo sia quello di trovare la media in diversi modi in base alla velocita.
import random
def average_py(n):
    s=0
    for i in range(n):
        s += random.random()
    return s/n

In [4]:
n=1000000
average_py(n)

0.49989778496279785

In [5]:
%timeit average_py(n)

92.9 ms ± 13.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%time sum([random.random() for _ in range(n)])/n

CPU times: total: 93.8 ms
Wall time: 114 ms


0.5004318588338118

In [7]:
#utilizzare numpy ci permettera di fare la stessa operazione piu velocemente per le sue capacità di vettorizzazione.
import numpy as np 
def average_np(n):
    s=np.random.random(n)
    return s.mean()
%time average_np(n)

CPU times: total: 0 ns
Wall time: 15.7 ms


0.5004680927395805

In [8]:
s=np.random.random(n)
s.nbytes

8000000

In [9]:
#numpy guadagna in velocità ma perde in memoria
#numba velocizza e risparmia.
import numba
average_nb = numba.jit(average_py)
%time average_nb(n)

C:\Users\matty\AppData\Local\Temp\ipykernel_16028\207230747.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  average_nb = numba.jit(average_py)


CPU times: total: 109 ms
Wall time: 801 ms


0.49979178114486356

In [11]:
#cython è piu veloce ma piu difficile da usare
%load_ext Cython
%%Cython -a
import random
def average_cy1(int n):
    cdef int i
    cdef float s = 0
    for i in range(n):
        s += random.random()
     return s/n

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)

In [ ]:
#l ultima funzione nn va


# algoritmi

In [ ]:
#prendiamo algoritmi usati per le performance di benchmark e facciamo es. performance come nella sezione precedente
#creiamo un algoritmo che ci permette di calcolare se un numero è primo o meno
def is_prime(I):
    if I % 2 ==0: return False #se è dispari ritorna falso
    for i in range(3, int(I**0.5)+1,2):
        if I % i==0:return False
    return True


In [ ]:
n=int(1e8 + 3)
n

In [ ]:
#con numba la stessa cosa la facciamo piu rapidamente
is_prime_nb = numba.jit(is_prime)

In [ ]:
#cython è ancora piu veloce, ma il codice è statico
%%cython
def is_prime_cy1(I):
    if I % 2 == 0:return False
    for i in range(3, int(I**0.5)+1,2):
        if I % i == 0: return False
    return True


In [ ]:
# in alcuni ambiti come nel caso dei numeri primi multipli numeri devono essere considerarti allo stesso tempo
#il multiprocessing serve quindi a fare proprio operazioni in maniera parallela
import multiprocessing as mp 


In [ ]:
pool = mp.Pool(processes=4)
%time pool.map(is_prime,10*[p1])
#precedentemente non ho definito p1

# Fibonacci numbers

In [ ]:
#anche fibonacci viene implementato tramite algoritmo
#iniziamo con due 1
#l algoritmo viene sempre messo a confronto con la velocità delle diverse librerie
def fib_rec_py1(n):
    if n < 2:
        return n
    else:
        return fib_rec_py1(n-1)+fib_rec_py1(n-2)

In [ ]:
fib_rec_py1(30)

In [ ]:
fib_rec_nb = numba.jit(fib_rec_py1)

In [ ]:
#con i codici scritti non calcoliamo/teniamo conto dei risultati intermedi, per questo:
from functools import lru_cache as cache
@cache(maxsize=None)
def fib_rec_py2(n):
        if n <2:
            return n
        else:return fib_rec_py2(n-1)+fib_rec_py2(n-2)

In [ ]:
%time fib_rec_py2(60)
#gia con questo metodo , è infinitamente piu veloce e non si blocca

In [ ]:
#usiamo l algoritmo interattivo
def fib_it_py(n):
    x,y = 0, 1
    for i in range(1, n+1):
        x, y = y, x + y
    return x


In [ ]:
%time fib_it_py(80)

In [ ]:
fn = fib_rec_py2(150)

In [ ]:
fn.bit_length()

# the number pi

In [ ]:
#analizziamo il pi greco, lo creiamo simulando i punti di una circonferenza coompresi in un area -1,1
import random
import numpy as np
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family']='serif'
%matplotlib inline

rn = [(random.random() * 2 - 1, random.random() * 2 - 1) 
      for _ in range(500)]
rn = np.array(rn)
rn

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(1,1,1)
circ = plt.Circle((0,0),radius = 1, edgecolor='g',lw=2.0, facecolor='None')
box = plt.Rectangle((-1,1),2,2,edgecolor='b', alpha=0.3)
ax.add_patch(circ)
ax.add_patch(box)
plt.plot(rn[:,0],rn[:,1],'r.')
plt.ylim(-1.1, 1.1)
plt.xlim(-1.1, 1.1)

In [ ]:
#ovviamente si puo scrivere con numpy o cython codici piu veloci


# BINOMIAL TREES

In [30]:
#lo applichiamo alla ross coss e rubinstein
import math
S0=36
T=1.0
r=0.06
sigma=0.2

def simulate_tree(M):
    dt=T/M
    u=math.exp(sigma *math.sqrt(dt))
    d=1/u
    S=np.zeros((M+1,M+1))
    S[0,0]=S0
    z=1
    for t in range(1,M+1):
        for i in range(z):
            S[i,t]=S[i,t-1]*u
            S[i+1,t]=S[i,t-1]*d
        z +=1
    return S

In [31]:
np.set_printoptions(formatter={'float': lambda x: '%6.2f' % x}) #rappresenta la tecnica di visualizzazione della matrice, 
#le virgole e la larghezza.
simulate_tree(4)

array([[ 36.00,  39.79,  43.97,  48.59,  53.71],
       [  0.00,  32.57,  36.00,  39.79,  43.97],
       [  0.00,   0.00,  29.47,  32.57,  36.00],
       [  0.00,   0.00,   0.00,  26.67,  29.47],
       [  0.00,   0.00,   0.00,   0.00,  24.13]])

In [32]:
%time simulate_tree(500)

CPU times: total: 31.2 ms
Wall time: 90.3 ms


array([[ 36.00,  36.32,  36.65,  36.98,  37.31,  37.65,  37.98,  38.33,
         38.67,  39.02,  39.37,  39.72,  40.08,  40.44,  40.80,  41.17,
         41.54,  41.91,  42.29,  42.67,  43.05,  43.44,  43.83,  44.22,
         44.62,  45.02,  45.43,  45.83,  46.25,  46.66,  47.08,  47.50,
         47.93,  48.36,  48.79,  49.23,  49.68,  50.12,  50.57,  51.03,
         51.48,  51.95,  52.41,  52.89,  53.36,  53.84,  54.32,  54.81,
         55.30,  55.80,  56.30,  56.81,  57.32,  57.83,  58.35,  58.88,
         59.41,  59.94,  60.48,  61.02,  61.57,  62.12,  62.68,  63.24,
         63.81,  64.39,  64.96,  65.55,  66.14,  66.73,  67.33,  67.94,
         68.55,  69.16,  69.78,  70.41,  71.04,  71.68,  72.33,  72.97,
         73.63,  74.29,  74.96,  75.63,  76.31,  77.00,  77.69,  78.39,
         79.09,  79.80,  80.52,  81.24,  81.97,  82.71,  83.45,  84.20,
         84.96,  85.72,  86.49,  87.27,  88.05,  88.84,  89.64,  90.45,
         91.26,  92.08,  92.91,  93.74,  94.59,  95.43,  96.29, 

In [33]:
#quest ultima matrice è strana e forse non utile.
#si puo partire dal codice precedente per implemetare un albero usando numpy.pag 296.

In [34]:
#oppure si puo proprio creare un codice numpy compatto e vettorizzato partendo da zero.
def simulate_tree_np(M):
    dt=T/M
    up=np.arange(M+1)
    up=np.resize(up,(M+1,M+1))
    down=up.transpose()*2
    S=S0*np.exp(sigma*math.sqrt(dt)*(up-down))
    return S

In [35]:
simulate_tree_np(4)

array([[ 36.00,  39.79,  43.97,  48.59,  53.71],
       [ 29.47,  32.57,  36.00,  39.79,  43.97],
       [ 24.13,  26.67,  29.47,  32.57,  36.00],
       [ 19.76,  21.84,  24.13,  26.67,  29.47],
       [ 16.18,  17.88,  19.76,  21.84,  24.13]])

In [36]:
#la stessa cosa si puo fare con numba
simulate_tree_nb = numba.jit(simulate_tree)
simulate_tree_nb(4)

C:\Users\matty\AppData\Local\Temp\ipykernel_16028\3691215025.py:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  simulate_tree_nb = numba.jit(simulate_tree)


array([[ 36.00,  39.79,  43.97,  48.59,  53.71],
       [  0.00,  32.57,  36.00,  39.79,  43.97],
       [  0.00,   0.00,  29.47,  32.57,  36.00],
       [  0.00,   0.00,   0.00,  26.67,  29.47],
       [  0.00,   0.00,   0.00,   0.00,  24.13]])

In [37]:
#infine troviamo cython che nn uso


# Monte carlo simulation


In [38]:
#M sono gli intervalli di tempo
#implementiamo una simulazione montecarlo con sottostante un azione
#successivamente la implementeremo con altri metodi
M=100 #numero di intervalli di discretizzazione
I=50000 #numero simulazioni
def mcs_symulation_py(p):
    M,I=p
    dt=T/M
    S=np.zeros((M+1,I))
    S[0]=S0
    rn=np.random.standard_normal(S.shape) #numeri casuali generati in un unico vettore
    for t in range(1,M+1): #evoluzione di St basata sullo schema di eulero
        for i in range(I):#
            S[t,i]=S[t-1,i]*math.exp((r-sigma**2/2)*dt+sigma*math.sqrt(dt)*rn[t,i])#
    return S


In [39]:
%time S=mcs_symulation_py((M,I))


CPU times: total: 4.17 s
Wall time: 5.36 s


In [40]:
print(S)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
np.set_printoptions(threshold=np.inf)  # Imposta la visualizzazione completa della matrice
print(S)

In [ ]:
S[-1].mean()#la media alla fine del periodo basata sulla simulazione


In [ ]:
S0*math.exp(r*T)#valore atteso alla fine del periodo
K=40

In [ ]:
C0=math.exp(-r*T)*np.maximum(K-S[-1],0).mean()#stimatore monte carlo per l opzionedef

In [ ]:
C0

In [ ]:
#facciamo simulazione con NumPy
def mcs_symulation_np(p):
    M,I=p
    dt=T/M
    S=np.zeros((M+1,I))
    S[0]=S0
    rn=np.random.standard_normal(S.shape)
    for t in range(1,M+1):
        S[t]=S[t-1]*np.exp((r-sigma**2/2)*dt+sigma*math.sqrt(dt)*rn[t])
    return S

In [ ]:
%time S=mcs_symulation_np((M,I))

In [ ]:
#infine con nunba che è ancora piu veloce
mcs_symulation_nb = numba.jit(mcs_symulation_py)
%time S=mcs_symulation_nb((M,I))

In [ ]:
S[-1].mean()
C0=math.exp(-r*T)*np.maximum(K-S[-1],0).mean()

In [ ]:
C0

# multiprocessing simulation


In [ ]:
import multiprocessing as mp
pool = mp.Pool(processes=4)
p=20


In [ ]:
%time S=np.hstack(pool.map(mcs_symulation_np ,p*[(M,int(I/p))]))